In [34]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [35]:
fileName = "C:\GitHubRepos\ApsimX\Tests\Validation\Wheat\Maricopa\MARICOPA Wheat FACE data_2024-10-22 (ICASA data format v4.1)(PM3)(BAK1)(no soil temp).xlsx"

In [36]:
NameMapExcel = pd.read_excel("C:\GitHubRepos\ApsimX\Tests\Validation\Wheat\Maricopa\SimNameMap.xlsx")
NameMap = dict(zip(NameMapExcel.Code.values,NameMapExcel.SimulationName.values))
VariableMapExcel = pd.read_excel("C:\GitHubRepos\ApsimX\Tests\Validation\Wheat\Maricopa\VariableNameMap.xlsx")
VariableMapExcel.set_index('AGMIPName',inplace=True)
VariableMapExcel.drop_duplicates(inplace=True)
VarMap = dict(zip(VariableMapExcel.index.values,VariableMapExcel.Renamed))

In [37]:
Harvest= pd.read_excel(fileName,sheet_name='Obs_crop_summary_plots',skiprows=[1,2])
Harvest.columns = [VarMap[x] for x in Harvest.columns]
Harvest.loc[:,'SimulationName']= [NameMap[x] for x in Harvest.loc[:,'treatment_ID']]
Harvest.set_index(['SimulationName','Clock.Today'],inplace=True)
Harvest.loc[:,'Wheat.Phenology.AnthesisDAS'] = Harvest.anthesis_date-Harvest.planting_date
Harvest.loc[:,'Wheat.Phenology.MaturityDAS'] = Harvest.physiologic_maturity_dat-Harvest.planting_date

HarvFilter = (VariableMapExcel.Timing=="Harvest")&(VariableMapExcel.InAPSIM==1)
HarvestReportVars = VariableMapExcel.loc[HarvFilter,:].Renamed.values
HarvestReports = Harvest.reindex(HarvestReportVars,axis=1)
HarvestConversions=dict(zip(VariableMapExcel.loc[HarvFilter,:].Renamed,VariableMapExcel.loc[HarvFilter,:].Conversion))
for col in HarvestReports.columns:
    HarvestReports.loc[:,col] = HarvestReports.loc[:,col] / HarvestConversions[col]
HarvestFinal = HarvestReports.groupby(level=[0,1]).mean().join(HarvestReports.groupby(level=[0,1]).sem(),rsuffix='Error')
HarvestFinal.loc[:,'Wheat.Phenology.CurrentStageName'] = "HarvestRipe"

In [38]:
Daily= pd.read_excel(fileName,sheet_name='Obs_crop_daily_plots',skiprows=[1,2])
Daily.columns = [VarMap[x] for x in Daily.columns]
Daily.loc[:,'SimulationName']= [NameMap[x] for x in Daily.loc[:,'Treatment ID']]
Daily.set_index(['SimulationName','Clock.Today'],inplace=True)
DailyFilter = (VariableMapExcel.Timing=="Daily")&(VariableMapExcel.InAPSIM==1)
DailyReportVars = VariableMapExcel.loc[DailyFilter,:].Renamed.values
DailyReports = Daily.reindex(DailyReportVars,axis=1)
DailyConversions=dict(zip(VariableMapExcel.loc[DailyFilter,:].Renamed,VariableMapExcel.loc[DailyFilter,:].Conversion))
for col in DailyReports.columns:
    DailyReports.loc[:,col] = DailyReports.loc[:,col] / DailyConversions[col]
DailyFinal = DailyReports.groupby(level=[0,1]).mean().join(DailyReports.groupby(level=[0,1]).sem(),rsuffix='Error')

In [39]:
AllData = pd.concat([HarvestFinal,DailyFinal])
AllData.loc[:,"Wheat.Ear.Wt"] = AllData.loc[:,"Wheat.Grain.Wt"] + AllData.loc[:,"Wheat.Spike.Wt"]
AllData.loc[:,"Wheat.Ear.N"] = AllData.loc[:,"Wheat.Grain.N"] + AllData.loc[:,"Wheat.Spike.N"]

In [40]:
SWCRaw= pd.read_excel(fileName,sheet_name='Soil_moisture_plots',skiprows=[1,2])
SWCRaw.loc[:,'SimulationName']= [NameMap[x] for x in SWCRaw.loc[:,'treatment_ID']]
SWCRaw.columns = ['experiment_ID', 'treatment_ID', 'Replicate', 'Clock.Today',
       'Soil.Depth', 'ObservedLayers.SW', 'SimulationName']

SoilLayers = dict(zip(SWCRaw.loc[:,'Soil.Depth'].drop_duplicates().values,range(1,11)))

SWCRaw.loc[:,'Layer'] = [SoilLayers[x] for x in SWCRaw.loc[:,'Soil.Depth']]
SWCunstacked = SWCRaw.pivot(columns='Layer',index=['SimulationName','Clock.Today','Replicate'],values='ObservedLayers.SW')
SWCunstacked.index =    SWCunstacked.index.droplevel(2)
SWCunstacked.columns = ['ObservedLayers.SW('+str(x)+')' for x in SWCunstacked.columns]

def calcProfileSWC(layers):
    lbind = range(len(layers))
    profileSum = 0
    for l in lbind:
        if l == 0:
            profileSum += layers[l] * 300
        else:
            profileSum += layers[1] * 200
    return profileSum

SWCunstacked.loc[:,'sum(ObservedLayers.SWmm)']=[calcProfileSWC(SWCunstacked.iloc[x,:].values) for x in range(SWCunstacked.index.size)]
SWCMean = SWCunstacked.groupby(level=[0,1]).mean()
SWCSEM = SWCunstacked.groupby(level=[0,1]).sem()
AllSWC = SWCMean.join(SWCSEM,rsuffix='Error')
AllData = pd.concat([AllData,AllSWC])

In [41]:
PARRaw = pd.read_excel(fileName,sheet_name='PAR_plots',skiprows=[1,2])
PARRaw.loc[:,'SimulationName']= [NameMap[x] for x in PARRaw.loc[:,'treatment_ID']]
PARRaw.columns = ['experiment_ID', 'treatment_ID', 'Clock.Today',
       'time_of_measurment', 'Replicate', 'PAR_incident_noon',
       'PAR_reflected_by_canopy_noon', 'frac_PAR_reflected_by_canopy_noon',
       'PAR_transmitted_by_canopy_noon', 'frac_PAR_transmitted_by_canopy_noon',
       'PAR_reflected_by_soil_noon', 'frac_PAR_reflected_by_soil_noon',
       'PAR_absorbed_canopy_noon', 'Wheat.Leaf.CoverTotal',
       'effec_frac_PAR_absorbed_canopy_noon', 'Wheat.Leaf.LAI',
       'stem_area_index', 'norm_diff_veg_index', 'SimulationName']
PAR = PARRaw.reindex(['SimulationName', 'Clock.Today', 'Wheat.Leaf.CoverTotal','Wheat.Leaf.LAI'],axis=1).set_index(['SimulationName', 'Clock.Today'])
AllPAR = PAR.groupby(level=[0,1]).mean().join(PAR.groupby(level=[0,1]).sem(),rsuffix='Error')
AllData = pd.concat([AllData,AllPAR])

In [42]:
NDVIRaw = pd.read_excel(fileName,sheet_name='NDVI_plots',skiprows=[1,2])
NDVIRaw.loc[:,'SimulationName']= [NameMap[x] for x in NDVIRaw.loc[:,'treatment_ID']]
NDVIRaw.columns = ['experiment_ID', 'treatment_ID', 'Clock.Today',
       'time_of_measurment', 'Replicate', 'flag_time_series_data',
       'near_infrared_reflectance', 'ratio_veg_index', 'NDVIModel.Scrip.NDVI',
       'SimulationName']
NDVI = NDVIRaw.reindex(['SimulationName','Clock.Today','NDVIModel.Script.NDVI'],axis=1).set_index(['SimulationName', 'Clock.Today'])
AllNDVI = NDVI.groupby(level=[0,1]).mean().join(NDVI.groupby(level=[0,1]).sem(),rsuffix='Error')
AllData = pd.concat([AllData,AllNDVI])

In [43]:
AllData.dropna(how='all',inplace=True)
AllData.to_excel("Maricopa.xlsx",sheet_name="Observed",merge_cells=False)

In [44]:
list(AllData.columns)

['Wheat.Phenology.AnthesisDAS',
 'Wheat.Phenology.MaturityDAS',
 'Wheat.Phenology.FinalLeafNumber',
 'Wheat.Leaf.Live.Wt',
 'Wheat.Leaf.Dead.Wt',
 'Wheat.Leaf.Wt',
 'Wheat.Stem.Wt',
 'Wheat.Spike.Wt',
 'Wheat.Grain.Wt',
 'Wheat.AboveGround.Wt',
 'Wheat.Leaf.Live.N',
 'Wheat.Leaf.Deat.N',
 'Wheat.Leaf.N',
 'Wheat.Stem.N',
 'Wheat.Spike.N',
 'Wheat.Grain.N',
 'Wheat.AboveGround.N',
 'Wheat.Grain.Number',
 'Wheat.Grain.Size',
 'Wheat.Grain.Protein',
 'Wheat.Phenology.AnthesisDASError',
 'Wheat.Phenology.MaturityDASError',
 'Wheat.Phenology.FinalLeafNumberError',
 'Wheat.Leaf.Live.WtError',
 'Wheat.Leaf.Dead.WtError',
 'Wheat.Leaf.WtError',
 'Wheat.Stem.WtError',
 'Wheat.Spike.WtError',
 'Wheat.Grain.WtError',
 'Wheat.AboveGround.WtError',
 'Wheat.Leaf.Live.NError',
 'Wheat.Leaf.Deat.NError',
 'Wheat.Leaf.NError',
 'Wheat.Stem.NError',
 'Wheat.Spike.NError',
 'Wheat.Grain.NError',
 'Wheat.AboveGround.NError',
 'Wheat.Grain.NumberError',
 'Wheat.Grain.SizeError',
 'Wheat.Grain.ProteinError'